In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from mysql.connector import (connection)
from collections import defaultdict
cnx = connection.MySQLConnection(user='root', password='samanJA1381',
                                 host='127.0.0.1',
                                 database='maktabkhooneh')


In [3]:
filt = (df['brand']=='BMW') & (df['year'] > 2020)
new_df = df.loc[filt]
new_df

,brand,model,year,price$,mileage
40,BMW,s,2021,25900,62813
58,BMW,s,2021,25900,62813
66,BMW,120,2021,30997,31700
70,BMW,120,2021,27998,17370
72,BMW,s,2021,34174,6731
...,...,...,...,...,...
70088,BMW,i3s,2021,25900,62813
70116,BMW,i3120,2021,30997,31700
70130,BMW,i3120,2021,27998,17370
70139,BMW,i3s,2021,34174,6731


In [4]:
df['price$'] = df['price$'].apply(lambda x: ('%i$' % x))

In [5]:
df.columns = df.columns.str.capitalize()

In [6]:
df

,Brand,Model,Year,Price$,Mileage
0,Nissan,VersaSV,2021,11995$,63150
1,Ford,ExplorerXLT,2020,19900$,94741
2,Nissan,VersaSV,2021,14598$,64097
3,Chevrolet,EquinoxLT,2019,14367$,77349
4,Chevrolet,EquinoxLT,2020,16698$,78322
...,...,...,...,...,...
70384,Lamborghini,HuracanSpyder,2017,230000$,9121
70385,Lamborghini,HuracanLP,2015,199991$,21854
70386,Lamborghini,HuracanSpyder,2020,284900$,4211
70387,Lamborghini,HuracanSTO,2023,444841$,3409


In [12]:
df.drop_duplicates(inplace=True)

In [14]:
df.reset_index()

,index,Brand,Model,Year,Price$,Mileage
0,0,Nissan,VersaSV,2021,11995$,63150
1,1,Ford,ExplorerXLT,2020,19900$,94741
2,2,Nissan,VersaSV,2021,14598$,64097
3,3,Chevrolet,EquinoxLT,2019,14367$,77349
4,4,Chevrolet,EquinoxLT,2020,16698$,78322
...,...,...,...,...,...,...
256,27728,Lamborghini,HuracanCoupe,2017,229998$,12844
257,28024,Lamborghini,HuracanSpyder,2017,229991$,30452
258,28321,Lamborghini,HuracanPerformante,2018,359998$,8776
259,28619,Lamborghini,HuracanLP,2016,199998$,33844


In [16]:
filt = df['Brand'] == 'Lamborghini'
lambo = df.loc[filt]

In [17]:
lambo.reset_index()

,index,Brand,Model,Year,Price$,Mileage
0,178,Lamborghini,HuracanSTO,2023,375500$,1693
1,358,Lamborghini,HuracanSpyder,2017,249756$,1428
2,539,Lamborghini,HuracanCoupe,2020,279596$,5512
3,721,Lamborghini,HuracanSTO,2022,389900$,172
4,904,Lamborghini,HuracanCoupeExcellent,2023,329988$,151
...,...,...,...,...,...,...
116,27728,Lamborghini,HuracanCoupe,2017,229998$,12844
117,28024,Lamborghini,HuracanSpyder,2017,229991$,30452
118,28321,Lamborghini,HuracanPerformante,2018,359998$,8776
119,28619,Lamborghini,HuracanLP,2016,199998$,33844
